In [161]:
import investpy
import pandas as pd

## Fetching and formatting the data
Below you will find the *investpy* library in use, which allows us to find ETF stocks based on their ISIN number

In [162]:
def get_recent_data_from_isins(isins):
    df = pd.DataFrame()

    for isin in isins:
        search_results = investpy.search_quotes(text=isin, countries=['germany'])
        data = search_results[0].retrieve_historical_data(from_date='01/01/2010', to_date='11/08/2020')
        data['Name'] = search_results[0].name
        data['ISIN'] = isin
        df = pd.concat([df, data])
    df.sort_index(inplace=True)
    return df

In [163]:
isins = [
    'LU1681043599', 
    'IE00B86MWN23', 
    'LU0533033667', 
    'LU1650490474',
    ]

etf_dataset = get_recent_data_from_isins(isins)

## What do the results look like?
Here we can see that our data has succesfully loaded, now we can get to analyzing!

In [164]:
etf_dataset

,Open,High,Low,Close,Volume,Name,ISIN
Date,,,,,,,
2010-09-30,63.420,63.42,63.120,63.120,40,Lyxor MSCI World Information Technology TR UCI...,LU0533033667
2010-10-01,62.760,62.76,62.760,62.760,0,Lyxor MSCI World Information Technology TR UCI...,LU0533033667
2010-10-04,62.200,62.20,62.200,62.200,0,Lyxor MSCI World Information Technology TR UCI...,LU0533033667
2010-10-06,62.510,62.51,62.510,62.510,0,Lyxor MSCI World Information Technology TR UCI...,LU0533033667
2010-10-07,62.450,62.45,62.450,62.450,0,Lyxor MSCI World Information Technology TR UCI...,LU0533033667
...,...,...,...,...,...,...,...
2020-08-10,195.670,196.18,195.670,196.120,380,Lyxor UCITS EuroMTS All-Maturity Investment Gr...,LU1650490474
2020-08-11,44.785,45.15,44.785,44.865,10828,iShares Edge MSCI Europe Min Volatility UCITS,IE00B86MWN23
2020-08-11,302.300,303.19,301.630,302.440,3534,Amundi Index Solutions - Amundi MSCI World UCI...,LU1681043599


## Visualizing the results
Using *Bokeh* we can create interactive diagrams which show us how the ETF prices have developed over our given time.

In [165]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, NumeralTickFormatter

In [166]:
output_notebook()

Loading BokehJS ...

In [167]:
# Color generator for linechart, apparently not built into Bokeh
from bokeh.palettes import Category10
import itertools

def color_gen():
    yield from itertools.cycle(Category10[10])
color = color_gen()

In [168]:
def resample_etf_data(dataset, period_size, group='ISIN'):
    """
    Returns a resampled DataFrame for ETF Data.
    As a group, ISIN is used as a default, but can be adjusted if desired (e.g. Industry Category)

    Keyword arguments:
    dataset -- DataFrame containing ETF data
    period_size -- Pandas Resample rule, see pandas.DataFrame.resample for options
    group -- Group across which to repeat resampling (default 'ISIN')
    """

    resample_df = dataset.groupby(group).resample(period_size).mean()
    resample_df['pct_change'] = resample_df.groupby(group)['Close'].pct_change(periods=1).fillna(0)
    resample_df['abs_change'] = resample_df.groupby(group)['Close'].diff(periods=1).fillna(0)
    resample_df.reset_index(inplace=True)
    resample_df.set_index('Date')
    resample_df[group] = resample_df[group].astype('category')
    return resample_df

dataset = resample_etf_data(etf_dataset, 'M')

dataset


,ISIN,Date,Open,High,Low,Close,Volume,pct_change,abs_change
0,IE00B86MWN23,2014-10-31,31.872500,31.948750,31.821250,31.927500,7832.000000,0.000000,0.000000
1,IE00B86MWN23,2014-11-30,32.979500,33.073000,32.912500,33.005500,5262.000000,0.033764,1.078000
2,IE00B86MWN23,2014-12-31,33.362632,33.444737,33.157895,33.270000,36303.315789,0.008014,0.264500
3,IE00B86MWN23,2015-01-31,34.986190,35.195238,34.798095,35.021429,35499.476190,0.052643,1.751429
4,IE00B86MWN23,2015-02-28,37.029500,37.178000,36.945500,37.105500,26288.850000,0.059508,2.084071
...,...,...,...,...,...,...,...,...,...
317,LU1681043599,2020-04-30,267.313000,269.245000,264.477500,267.073500,8708.050000,0.050275,12.784409
318,LU1681043599,2020-05-31,281.726000,283.349500,280.076500,282.015500,4642.450000,0.055947,14.942000
319,LU1681043599,2020-06-30,292.540952,294.722381,290.713333,292.761429,5538.238095,0.038104,10.745929
320,LU1681043599,2020-07-31,296.950000,298.058261,295.272609,296.649130,3311.521739,0.013279,3.887702


In [169]:
def visualize_etf_data(dataset, x='Date', y='pct_change', group='ISIN', percentage=True):
    p = figure(plot_width=800, plot_height=400, x_axis_type='datetime')
    group_names = [value for value in dataset[group].unique()]

    for name in group_names:
        p.line(x, y, source=dataset[dataset[group] == name], legend_group=group, color=next(color), line_width=2)

    p.legend.location = 'bottom_left'
    if percentage:
        p.yaxis.formatter = NumeralTickFormatter(format='0 %')
        p.yaxis.axis_label = 'Value Change (%)'
        p.title.text = 'ETF Percentage Change over Time'
    else:
        p.title.text = 'ETF Value Change over Time'
        p.yaxis.axis_label = 'Value Change (€)'
        p.yaxis.formatter = NumeralTickFormatter(format='0.00')
    show(p)

    

visualize_etf_data(dataset)
visualize_etf_data(dataset, y='abs_change', percentage=False)

## What type of ETFs are we looking at?
Below you will find an example of an API called based on an ISIN number to show various info, including the spread across sectors (could also include countries/markets).

In [170]:
import httpx

descriptions = []

for isin in isins:
    r = httpx.get(f'https://api.etf-data.com/product/{isin}').json()
    descriptions.append({
        'name': r['name'],
        'totalFee': r['totalFee'],
        'distributionType': r['distributionType'],
        'baseCurrency': r['baseCurrency'],
        'sectors': r['sectors']
    })

descriptions_df = pd.DataFrame(descriptions)


In [171]:
from bokeh.palettes import Category20
from bokeh.transform import cumsum
from math import pi

for row in descriptions_df.itertuples():
    name = row.name
    sectors = {}
    for sector in row.sectors:
        sectors[sector['sector']] = sector['percentage']
        
    data = pd.Series(sectors).reset_index(name='value').rename(columns={'index':'sector'})
    data['angle'] = data['value']/data['value'].sum() * 2*pi
    # Todo: fix this silly color thing because it breaks when there's less than 3
    data['color'] = Category20[len(sectors)] if len(sectors) > 2 else 'blue'
    
    p = figure(plot_height=450, plot_width=800, title=f'Sectors: {name}', toolbar_location=None,
        tools="hover", tooltips="@sector: @value")
    
    p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_group='sector', source=data)
    p.legend.location = "top_left"
    p.legend.label_text_font_size = "6pt"
    show(p)
    print(data[['sector', 'value']])

                    sector  value
0   INFORMATION_TECHNOLOGY  21.95
1              HEALTH_CARE  13.42
2   CONSUMER_DISCRETIONARY  12.05
3               FINANCIALS  11.93
4              INDUSTRIALS  10.34
5   COMMUNICATION_SERVICES   9.21
6         CONSUMER_STAPLES   8.10
7                MATERIALS   4.43
8                UTILITIES   3.40
9              REAL_ESTATE   2.79
10                  ENERGY   2.38


                   sector  value
0        CONSUMER_STAPLES  19.03
1             HEALTH_CARE  15.81
2             INDUSTRIALS  13.13
3              FINANCIALS  12.89
4               UTILITIES  10.39
5  COMMUNICATION_SERVICES   8.36
6               MATERIALS   7.14
7  CONSUMER_DISCRETIONARY   5.22
8             REAL_ESTATE   4.24
9  INFORMATION_TECHNOLOGY   2.41


                   sector  value
0  INFORMATION_TECHNOLOGY  99.74
1  CONSUMER_DISCRETIONARY   0.14
2             INDUSTRIALS   0.12


       sector  value
0  GOVERNMENT  100.0
